In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from igraph import *
from math import *
import sys
import os
import glob
import random
from random import shuffle
from random import seed
import time
import datetime
import collections

In [14]:
import networkx as nx

In [4]:
graph = {}
with open('train_ab.txt') as inp:
    for l in inp.readlines():
        graph[l.strip().split("\t")[0]] = l.strip().split("\t")[1:]

In [6]:
with open('sample.txt','w') as outfile:
    for key in graph.keys():
        for val in graph[key]:
            outfile.write(f'{int(key)} {val}\n')

In [11]:
df = pd.read_csv('sample.txt',sep=" ",  header = None)
df.columns = ["source", "sink"]

In [12]:
df.head()

,source,sink
0,4066935,1272125
1,4066935,3105725
2,4066935,2828522
3,4066935,4394015
4,4066935,2367409


In [13]:
#saving the train data by removing the headers and indexes
df.to_csv('data/train_woheader.csv',header=False,index=False)
#storing the list of edges in a varible 
g=nx.read_edgelist('data/train_woheader.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
#printing the information of graph
print(nx.info(g))

FileNotFoundError: [Errno 2] No such file or directory: 'data/train_woheader.csv'

In [8]:
# load edge-list from file
def get_edge_list(dataset_path):
    data_file = open(dataset_path)
    edge_list = map(lambda x:tuple(map(int,x.split())),data_file.read().split("\n")[:-1])
    data_file.close()
    return list(edge_list)

In [12]:
dataset_path = 'sample.txt'

In [13]:
edge_list = get_edge_list(dataset_path)

In [14]:
# Calculates accuracy metrics (Precision & Recall),
# for a given similarity-model against a test-graph.
def print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set):
    precision = recall = c = 0
    for i in test_vertices_set:
        if i in train_vertices_set:
            actual_friends_of_i = set(test_graph.neighbors(i))

            # Handles case where test-data < k
            if len(actual_friends_of_i) < k:
                k2 = len(actual_friends_of_i)
            else:
                k2 = k

            top_k = set(get_top_k_recommendations(train_graph,sim,i,k2))

            precision += len(top_k.intersection(actual_friends_of_i))/float(k2)
            recall += len(top_k.intersection(actual_friends_of_i))/float(len(actual_friends_of_i))
            c += 1
    print("Precision is : " + str(precision/c))
    print("Recall is : " + str(recall/c))

In [15]:
# Get the similarity product for a path
# (product of path-step similarities)
def get_sim_product(sim, shortest_path):
    prod = 1
    for i in range(len(shortest_path) - 1):
        prod *= sim[shortest_path[i]][shortest_path[i+1]]
    return round(prod,3)

# Filter out, Sort and Get top-K predictions
def get_top_k_recommendations(graph,sim,i,k):
    return  sorted(filter(lambda x: i!=x and graph[i,x] != 1,range(len(sim[i]))) , key=lambda x: sim[i][x],reverse=True)[0:k]

# Convert edge_list into a set of constituent edges
def get_vertices_set(edge_list):
    res = set()
    for x,y in edge_list:
        res.add(x)
        res.add(y)
    return res

# Split the dataset into two parts (50-50 split)
# Create 2 graphs, 1 used for training and the other for testing
def split_data(edge_list):
    random.seed(350)
    #indexes = range(len(list(edge_list)))
    indexes = range(21)
    #test_indexes = set(random.sample(indexes, len(indexes)/2)) # removing 50% edges from test data
    test_indexes = set(random.sample(indexes, int(len(indexes)/2)))
    train_indexes = set(indexes).difference(test_indexes)
    test_list = [edge_list[i] for i in test_indexes]
    train_list = [edge_list[i] for i in train_indexes]
    return train_list,test_list

In [16]:
def local_methods(edge_list,method):
    train_list, test_list = split_data(edge_list)
    train_graph = Graph(train_list)
    test_graph = Graph(test_list)
    train_n =  train_graph.vcount() # This is maximum of the vertex id + 1
    train_vertices_set = get_vertices_set(train_list) # Need this because we have to only consider target users who are present in this train_vertices_set
    test_vertices_set = get_vertices_set(test_list) # Set of target users

    sim = [[0 for i in range(train_n)] for j in range(train_n)]
    for i in range(train_n):
        for j in range(train_n):
            if i!=j and i in train_vertices_set and j in train_vertices_set:
                sim[i][j] = similarity(train_graph,i,j,method)

    print_precision_and_recall(sim,train_graph,test_graph,test_vertices_set,train_vertices_set)

In [17]:
method = "common_neighbors"

In [ ]:
local_methods(edge_list,method)

In [11]:
len(list(edge_list))

21

In [72]:
range(913437)

range(0, 913437)

In [74]:
len(range(913437))

913437

In [91]:
list(edge_list)[0]

(4066935, 1272125)

In [63]:
edge_list

In [77]:
int(913437/2)

456718